# Исследование надежности заемщиков

## Цель проекта

Цель проекта: на основе статистики о платёжеспособности клиентов исследовать, влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок

## Описание данных

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

## Предобработка данных 

### Общая информация о датафрейме

1. Считываем файл в датафрейм

In [7]:
#импортируем библиотеку:
import pandas as pd
 
#с помощью конструкции try-except обрабатываем возможную ошибку при чтении файла:
try:
    data = pd.read_csv('C:/Users/user/Downloads/data_credit.csv')
except:
    data = pd.read_csv('/datasets/data.csv')

2. Выводим первые 10 строк на экран:

In [8]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


*Первое, что бросается в глаза, - отрицательные значения в столбце days_employed, в столбце education содержатся значения в верхнем и нижнем регистре, есть NaN в столбце total_income, который понадобится в дальнейшем для анализа*

3. Выводим общую информацию о датасете

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*Судя по информации о таблице, в столбцах days_employed и total_income есть много пропусков (19351 ненулевое значение при 21525 строк)*

### Удаление пропусков

Выводим количество пропусков по столбцам:

In [10]:
#метод isna() выдаст true в случае пропуска, а sum() эти true-значения сложит:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

*Пропуски в столбцах days_employed и total_income подтвердились*

Столбец — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом

In [11]:
#заполнение пропусков в total_income выполним в цикле
#этот цикл проходит по всем уникальным значениям столбца income_type и если в столбце total_income - пропуск, 
#заполняет его медианным значением по данному типу занятости income_type

for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()   

In [12]:
#проверим, что все ок:
data['total_income'].isna().sum()

0

Выбор медианы для заполнения пропусков характерными значениями обусловлено тем, что медиана "более устойчива" к аномальным значениям и выбросам, чем среднее арифметическое

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом является отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`

In [13]:
data['days_employed'] = data['days_employed'].abs()

In [14]:
#проверим, что все ок, выведем снова первые 10 строк датафрейма:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Для каждого типа занятости выведим медианное значение трудового стажа `days_employed` в днях:

In [15]:
#группируем по столбцу income_type, выбираем столбец days_employed и методом agg применяем к нему нахождение медианы
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их, как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`:

In [16]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Аномальное значение 20 скорее всего связано с опечаткой (случайно нажали 0), а вот -1 с клавиатуры ввести ошибочно сложно. Возможно, это признак ошибки при выгрузке данных. Удаляем данные с аномалиями:

In [17]:
#для того, чтобы удалить данные, просто исключаем их из выборки и перезаписываем датафрейм:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [18]:
#проверяем, что все ок
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

*Проверим наличие аномальных значений в других столбцах*

In [19]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [20]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

*В столбце много дубликатов по регистру, как и было замечено при первом знакомстве с данными. Чуть позже удалим их*

In [21]:
data['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [22]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [23]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [24]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

*В столбце присутствует аномальное значение - XNA. В рамках данного исследования столбец с данными о поле клиента нам не потребуется, поэтому пока оставим его без изменений*

In [25]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [26]:
data['debt'].unique()

array([0, 1], dtype=int64)

In [27]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

<div class="alert alert-info">Анализ уникальных значений столбцов датафрейма показал:<b></b></div>

    - в столбце gender присутствует артефакт XNA
    - наличие неявных дубликатов в столбце education
    - в столбце purpose присутствует опечатка (ремонт жилью)

*Исправим опечатку в столбце purpose, чтобы не было проблем при дальнейшей категоризации значений в этом столбце*

In [28]:
data.loc[data['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'

In [29]:
#проверим, что все ок
print(len(data.loc[data['purpose'] == 'ремонт жилью', 'purpose']))

0


Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`:

In [30]:
#заполнение пропусков в days_employed выполним в цикле
#этот цикл проходит по всем уникальным значениям столбца income_type и если в столбце days_employed - пропуск, 
#заполняет его медианным значением по данному типу занятости income_type

for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

*В столбце days_employed данные сильно ранжированы, поэтому для заполнения пропусков предпочтительнее использовать медианное значение*

Проверим, что все пропуски заполнены:

In [31]:
#проверяем, что все ок
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

*Пропуски в данных удалены. Аномальные значения в столбцах, необходимых для исследования, также исправлены*

### Изменение типов данных

Заменим вещественный тип данных на целое число (int) в столбце 'total_income'

In [32]:
data['total_income'] = data['total_income'].astype(int)

*Точностью до нескольких цифр после запятой в значении размера дохода клиента можно пренебречь*

### Обработка дубликатов 

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру

In [33]:
#для того, чтобы результаты метода str.lower сохранились, нужно перезаписать столбец

data['education'] = data['education'].str.lower()

In [34]:
#проверим результат:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверим, сколько имеется строк-дубликатов:

In [35]:
#метод duplicated() ищет дубликаты, а sum() их посчитает:

data.duplicated().sum()

71

In [36]:
data = data.drop_duplicates()

In [37]:
#проверим, что все ок:
data.duplicated().sum()

0

*Дубликаты в данных удалены, можно приступать к категоризации*

### Категоризация данных

В датафрейме создаем столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 назначаем категорию `'E'`, а клиенту, получающему 235000, — `'B'`

In [38]:
#данная функция возвращает категорию А, В, С, D, E в зависимости от значения переменной income
#конструкция try-except помогает обрабатывать исключения

def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [39]:
#результаты вызова функции categorize_income заносим в новый столбец total_income_category

data['total_income_category'] = data['total_income'].apply(categorize_income)

In [40]:
#проверим, что функция работает правильно, категория Е:
data.loc[(data['total_income'] < 30000),'total_income_category'].head(5)


934     E
1184    E
1598    E
4057    E
8273    E
Name: total_income_category, dtype: object

In [41]:
#проверим, что функция работает правильно, категория D:
data.loc[(data['total_income'] < 30000) & (data['total_income'] > 50000),'total_income_category'].head(5)

Series([], Name: total_income_category, dtype: object)

In [42]:
#проверим, что функция работает правильно, категория C:
data.loc[(data['total_income'] < 200000) & (data['total_income'] > 50000),'total_income_category'].head(5)

1    C
2    C
4    C
7    C
8    C
Name: total_income_category, dtype: object

In [43]:
#проверим, что функция работает правильно, категория B:
data.loc[(data['total_income'] < 1000000) & (data['total_income'] > 200000),'total_income_category'].head(5)

0     B
3     B
5     B
6     B
16    B
Name: total_income_category, dtype: object

In [44]:
#проверим, что функция работает правильно, категория A:
data.loc[(data['total_income'] > 1000000),'total_income_category'].head(5)

1590    A
2224    A
4129    A
4415    A
7447    A
Name: total_income_category, dtype: object

*В данные добавлена категоризация по уровню дохода клиента*

Создаем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.
Для группировки целей кредита на покупку автомобиля подойдет поиск подстроки 'автом', целиком слово 'автомобиля' использовать нельзя, так как у него меняется окончание. В случае операций с недвижимостью используются 2 слова - недвижимость и жилье. Их также нужно избавить от окончаний, 'недвиж' и 'жил' соответственно. В категории свадеб можно использовать подстроку 'свад', в образовании - 'образов'

In [45]:
#эта функция ищет подстроку в строке, выводит нужную категорию в случае если подстрока найдена
#для обработки исключений используется конструкция try-except, она вернет значение 'нет категории', если ничего не нашлось

def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [46]:
#в новый столбец purpose_category записываем результат вызова функции для столбца purpose

data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [47]:
#проверим, что функция отработала корректно, выведем первые 10 строк
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


*В данные добавлена категоризация по целям взятия кредита*

## Исследовательский анализ данных

### Зависимость между количеством детей и возвратом кредита в срок

*Для начала необходимо сгруппировать данные о количестве детей по категориям*

In [48]:
#проверим, как распределилась выборка по количеству детей
data['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Семьи по количеству детей, как правило, разделяются на следующие категории: бездетные (children = 0), 1 или 2 ребенка, многодетные (от 3 и более)

In [49]:
#создадим функцию, которая вернет нужную категорию в зависимости от значения столбца children
def categorize_children(row):
    try:
        if row == 0:
            return 'бездетные'
        elif row == 1:
            return '1 ребенок'
        elif row == 2:
            return '2 ребенка'
        elif row > 2:
            return 'многодетные'
    except:
        return 'нет категории'

In [50]:
#в новый столбец children_category записываем результат вызова функции для столбца children

data['children_category'] = data['children'].apply(categorize_children)

In [51]:
#проверим, что функция отработала правильно, выведем первые 10 строк 
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1 ребенок
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1 ребенок
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,бездетные
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетные
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,бездетные
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью,бездетные
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью,бездетные
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования,бездетные
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы,2 ребенка
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью,бездетные


In [54]:
#c помощью метода pivot_table подсчитаем сумму и общее количество просрочек по каждой группе клиентов 
#результатом будет сводная таблица data_pivot_children

data_pivot_children = data.pivot_table(index=['children_category'], values='debt', aggfunc= ['sum', 'count'])

#в новом столбце этой таблицы посчитаем долю просрочек

share_debt = data_pivot_children['sum'] / data_pivot_children['count'] * 100
data_pivot_children['share, %'] = round(share_debt, 2)

#отсортируем таблицу для наглядности восприятия
data_pivot_children.sort_values(by = 'share, %', ascending = True)

,sum,count,"share, %"
,debt,debt,
children_category,,,
бездетные,1063,14091,7.54
многодетные,31,380,8.16
1 ребенок,444,4808,9.23
2 ребенка,194,2052,9.45


<div class="alert alert-info"> <b>Вывод: </b>Анализ данных сводной таблицы показал, что наименьший уровень просрочки по кредитам у клиентов без детей, это имеет экономическое объяснение. Однако наибольший уровень выявлен у клиентов, имеющих 2 детей. Как это ни странно, многодетные клиенты по уровню просрочек чуть уступают бездетным.

### Зависимость между семейным положением и возвратом кредита в срок

*Семейное положение клиента содержит 4 уникальных значения, соответственно, дальнейшую категоризацию данных проводить не требуется, данные и так сгруппированы по основным типам. Можно сразу проходить к созданию сводной таблицы*

In [55]:
#c помощью метода pivot_table подсчитаем сумму и общее количество просрочек по каждой группе клиентов 
#результатом будет сводная таблица data_pivot_family

data_pivot_family = data.pivot_table(index=['family_status'], values='debt', aggfunc= ['sum', 'count'])

#в новом столбце этой таблицы посчитаем долю просрочек
share_debt_family = data_pivot_family['sum'] / data_pivot_family['count'] * 100
data_pivot_family['share, %'] = round(share_debt_family, 2)

#выводим результат, отсортировав по доле просрочек
data_pivot_family.sort_values(by = 'share, %', ascending = True)

,sum,count,"share, %"
,debt,debt,
family_status,,,
вдовец / вдова,63,951,6.62
в разводе,84,1189,7.06
женат / замужем,927,12261,7.56
гражданский брак,385,4134,9.31
Не женат / не замужем,273,2796,9.76


<div class="alert alert-info"> <b>Вывод: </b>Анализ данных сводной таблицы показал, что наименьший уровень просрочки по кредитам у вдовцов/вдов. Наибольший - у неженатых/не замужних клиентов. </div>

### Зависимость между уровнем дохода и возвратом кредита в срок

*Категоризацию по уровню дохода клиентов мы уже произвели на предыдущих этапах. Однако для большей наглядности результата создадим таблицу, которая будет возвращать описание категории*

In [56]:
#данная функция возвращает описание категории А, В, С, D, E 
#конструкция try-except помогает обрабатывать исключения

def describe_income(row):
    try:
        if row == 'E':
            return 'До 30000'
        elif row == 'D':
            return 'От 30000 до 50000'
        elif row == 'C':
            return 'От 50000 до 200000'
        elif row == 'B':
            return 'От 200000 до 1000000'
        elif row == 'A':
            return 'Свыше 1000000'
    except:
        pass

In [57]:
#добавим новый столбец с описанием дохода income_description и заполним его с помощью функции describe_income

data['income_description'] = data['total_income_category'].apply(describe_income)


*Проверять дополнительно отработку функции не будем, так как все будет видно в сводной таблице*

In [58]:
#c помощью метода pivot_table подсчитаем сумму и общее количество просрочек по каждой группе клиентов 
#результатом будет сводная таблица data_pivot_income
data_pivot_income = data.pivot_table(index=['total_income_category','income_description'], values='debt', aggfunc= ['sum', 'count'])

#в новом столбце этой таблицы посчитаем долю просрочек
share_debt_income = data_pivot_income['sum'] / data_pivot_income['count'] * 100
data_pivot_income['share, %'] = round(share_debt_income, 2)


#выводим результат на экран, отсортировав по доле просрочек
data_pivot_income.sort_values(by = 'share, %', ascending = True)


,,sum,count,"share, %"
,,debt,debt,
total_income_category,income_description,,,
D,От 30000 до 50000,21,349,6.02
B,От 200000 до 1000000,354,5014,7.06
A,Свыше 1000000,2,25,8.00
C,От 50000 до 200000,1353,15921,8.50
E,До 30000,2,22,9.09


<div class="alert alert-info"> <b>Вывод: </b>Анализ данных сводной таблицы показал, что наименьший уровень просрочки по кредитам у клиентов с доходом от 30000 до 50000 рублей. Наибольший уровень просрочки имеют клиенты с самым маленьким доходом до 30000, что имеет экономическое обоснование. Клиенты с самым высоким уровнем доходов имеют средний показатель уровня просрочки, что никак не коррелирует с экономикой. </div>

### Зависимость целей кредита на его возврат в срок

*Сначала проанализируем популярность среди целей кредита*

In [59]:
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

*Наибольшее количество кредитов берут на операции с недвижимостью, наименьшее - на проведение свадьбы*

In [61]:
#c помощью метода pivot_table подсчитаем сумму и общее количество просрочек по каждой категории целей 
#результатом будет сводная таблица data_pivot_purpose

data_pivot_purpose = data.pivot_table(index=['purpose_category'], values='debt', aggfunc= ['sum', 'count'])

#в новом столбце этой таблицы посчитаем долю просрочек
share_debt_purpose = data_pivot_purpose['sum'] / data_pivot_purpose['count'] * 100
data_pivot_purpose['share, %'] = round(share_debt_purpose)

#выводим результат на экран, отсортировав по доле просрочек
data_pivot_purpose.sort_values(by = 'share, %', ascending = True)

,sum,count,"share, %"
,debt,debt,
purpose_category,,,
операции с недвижимостью,780,10751,7.0
проведение свадьбы,183,2313,8.0
операции с автомобилем,400,4279,9.0
получение образования,369,3988,9.0


<div class="alert alert-info"> <b>Вывод: </b>Анализ данных сводной таблицы показал, что наименьший уровень просрочки имеют самые популярные кредиты на операции с недвижимостью. Наибольший уровень просрочки по кредитам - на операции с автомобилем.</div>

### Возможные причины появления пропусков в исходных данных

*При анализе данных были выявлены пропуски в столбцах days_employed и total_income. В days_employed имелись также отрицательные значения, а также большое ранжирование. Скорее всего, данный столбец заполняется автоматически, имеет смысл проверить логику работы системной процедуры, которая его заполняет. Уровень дохода клиента total_income не может заполняться автоматически, а также клиент не может оформить кредит, не указав свой доход. Поэтому появление пропусков в данном столбце скорее всего связано с ошибками выгрузки данных* 

## Общий вывод

<div class="alert alert-info"> <b> </b>   
    <br />На основе данных по статистике платежеспособности клиентов был проведен анализ зависимости разных факторов на возврат кредита в срок.Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.<br />
<br />Перед исследованием были выдвинуты следующие гипотезы:  <br />
<br />Гипотеза 1. Количество детей влияет на возврат кредита в срок<br />
<br />Гипотеза 2. Семейное положение влияет на погашение кредита в срок<br />
<br />Гипотеза 3. Уровень дохода влияет на возврат кредита в срок<br />
<br />Гипотеза 4. Цель кредита влияет на его погашение в срок<br />

<br /> В ходе анализа была определена доля просрочников по группам с разбивкой по количеству детей, семейному статусу, уровню дохода и целям взятия кредита.<br />
<br />В результате выявлены следующие зависимости.<br />
    
**Гипотеза 1: частично подтверждена, однако в рамках имеющихся данных делать вывод о наличии четкой зависимости нельзя.** 
<br />7.5% -  доля просроченных кредитов у клиентов без детей<br />
<br />8.2% -  доля просроченных кредитов среди многодетных клиентов<br />
<br />9.2% -  доля просроченных кредитов  у клиентов с 1 ребенком<br />
<br />9.5% -  доля просроченных кредитов у клиентов с 2 детьми<br />

По группе заемщиков, имеющих 3,4,5 детей, относительно мало данных. Рекомендуется провести дополнительное исследование на большей выборке данных.

**Гипотеза 2: подтверждена**
<br />6.6% - доля просроченных кредитов у клиентов-вдовцов<br />
<br />7.1% - доля просроченных кредитов у клиентов в разводе<br />
<br />7.6% - доля просроченных кредитов у женатых/замужних клиентов<br />
<br />9.3% - доля просроченных кредитов у клиентов в гражданском браке<br />
<br />9.8% - доля просроченных кредитов у неженатых/незамужних клиентов<br />

Разница в доле просрочки у клиентов в браке или когда-либо состоявщих в нем (вдовцы, в разводе, женатые) небольшая.
По сравнению с этими категориями клиенты, не зарегистрировавшие брак (в гражданском браке, неженатые), имеют долю просрочки выше на 2 пункта, что выделяет их на фоне общей выборки

**Гипотеза 3: частично подтверждена, однако в рамках имеющихся данных делать вывод о наличии четкой зависимости нельзя, для более точных выводов необходимы данные о клиентах с доходами категории D (от 30000 до 50000), A (свыше 1000000), E (до 30000)**
<br />6% - доля просроченных кредитов у клиентов с доходом категории D (от 30000 до 50000)<br />
<br />7.1% - доля просроченных кредитов  у клиентов с доходом категории B (от 200000 до 1000000)<br />
<br />8% - доля просроченных кредитов у клиентов с доходом категории А (свыше 1000000)<br />
<br />8.5% - доля просроченных кредитов у клиентов с доходом категории С (от 50000 до 200000)<br />
<br />9.1% - доля просроченных кредитов у клиентов с доходом категории Е (до 30000)<br />

Рекомендуется провести повторный анализ на большей выборке данных с клиентами категорий D, A, E

**Гипотеза 4: подтверждена**
<br />7.3% - доля просроченных кредитов на операции с недвижимостью<br />
<br />7.9% - доля просроченных кредитов на проведение свадьбы<br />
<br />9.25% - доля просроченных кредитов на получение образования<br />
<br />9.3% - доля просроченных кредитов на операции с автомобилем<br />


Таким образом, для построения модели кредитного скоринга можно использовать зависимости из 2 и 4 гипотезы. Также необходимо провести дополнительный анализ зависимости из гипотез 1 и 3 на более репрезентативной выборке данных.
    
</div>